# Mapping Tasiyagnunpa (Western Meadowlark) migration

Introduction to vector data operations

Tasiyagnunpa (Western Meadowlark, or sturnella neglecta) migrates each
year to next on the Great Plains in the United States. Using
crowd-sourced observations of these birds, we can see that migration
happening throughout the year.

> Read more about the Lakota connection to Tasiyagnunpa from [Native Sun
> News
> Today](https://www.nativesunnews.today/articles/meadowlarks-still-speak-lakota-humans-dont-anymore/)

## Import Python libraries

In [1]:
import calendar
import os
import pathlib
import requests
import ssl

import cartopy.crs as ccrs
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import pandas as pd
import tqdm

## Create a folder for your data

In [2]:
# Create data directory
data_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'species-distribution',
)
os.makedirs(data_dir, exist_ok=True)

gbif_path_tmpl = os.path.join(data_dir, 'meadowlark_observations_{month:02d}.csv')

## Define your study area

Track observations of Taciyagnunpa across the different **ecoregions**
of North America! You should be able to see changes in the number of
observations in each ecoregion throughout the year.

### Download and save ecoregion boundaries

> **<i class="fa fa-solid fa-keyboard fa-large" aria-label="keyboard"></i>
> Your Task**
>
> 1.  Find the URL for for the level III ecoregion boundaries. You can
>     [get ecoregion boundaries from the Environmental Protection Agency
>     (EPA).](https://www.epa.gov/eco-research/ecoregions-north-america).
> 2.  Replace `your/url/here` with the URL you found, making sure to
>     format it so it is easily readable.
> 3.  Change all the variable names to **descriptive** variable names
> 4.  Run the cell to download and save the data.

In [3]:
# Set up the ecoregions level III boundary URL
a_url = ("your/url/here")
# Set up a path to save the dataon your machine
a_path = os.path.join(data_dir, 'filename.zip')

# Don't download twice
if not os.path.exists(a_path):
    # Download, and don't check the certificate for the EPA
    a_response = requests.get(a_url, verify=False)
    # Save the binary data to a file
    with open(a_path, 'wb') as a_file:
        a_file.write(a_response.content)

### Load the ecoregions into Python

> **<i class="fa fa-solid fa-keyboard fa-large" aria-label="keyboard"></i>
> Your task**
>
> Download and save ecoregion boundaries from the EPA:
>
> 1.  Replace `a_path` with the path your created for your ecoregions
>     file.
> 2.  (optional) Consider renaming and selecting columns to make your
>     `GeoDataFrame` easier to work with.
> 3.  Make a quick plot with `.plot()` to make sure the download worked.
> 4.  Run the cell to load the data into Python

In [5]:
# Open up the ecoregions boundaries
gdf = gpd.read_file(a_path)

# Name the index so it will match the other data later on
gdf.index.name = 'ecoregion'

# Plot the ecoregions to check download

### Create a simplified `GeoDataFrame` for plotting

Plotting larger files can be time consuming. The code below will
streamline plotting with `hvplot` by simplifying the geometry,
projecting it to a Mercator projection that is compatible with
`geoviews`, and cropping off areas in the Arctic.

> **<i class="fa fa-solid fa-keyboard fa-large" aria-label="keyboard"></i>
> Your task**
>
> Download and save ecoregion boundaries from the EPA:
>
> 1.  Make a copy of your ecoregions `GeoDataFrame` with the `.copy()`
>     method, and save it to another variable name. Make sure to do
>     everything else in this cell with your new copy!
> 2.  Simplify the ecoregions with `.simplify(1000)`, and save it back
>     to the `geometry` column.
> 3.  Change the Coordinate Reference System (CRS) to Mercator with
>     \`.to_crs(ccrs.Mercator())
> 4.  Use the plotting code in the cell to check that the plotting runs
>     quickly and looks the way you want, making sure to change `gdf` to
>     YOUR `GeoDataFrame` name.

In [7]:
# Make a copy of the ecoregions

# Simplify the geometry to speed up processing

# Change the CRS to Mercator for mapping

# Check that the plot runs
gdf.hvplot(geo=True, crs=ccrs.Mercator())

## Download occurrence data

You will use a database called the [Global Biodiversity Information
Facility (GBIF)](https://www.gbif.org/). GBIF is compiled from species
observation data all over the world, and includes everything from museum
specimens to photos taken by citizen scientists in their backyards.

> **<i class="fa fa-solid fa-keyboard fa-large" aria-label="keyboard"></i>
> Your task**
>
> Download data from GBIF:
>
> 1.  Add parameters to the GBIF download function to limit your query

In [9]:
# Download meadowlark occurrence observations from GBIF
gbif_api_endpoint = 'https://api.gbif.org/v1/occurrence/search'

# Some of the columns are not present in all records
# Extract the ones we need so the file loads without errors
columns = [
    'key', 'occurrenceID', 'basisOfRecord', 'occurrenceStatus',
    'genus', 'species', 
    'decimalLatitude', 'decimalLongitude', 'continent', 
    'eventDate', 'month'
]

gbif_paths = []
# We need to download for each month to avoid the record limit
for month in range(1, 13):
    # Format monthly path
    gbif_path = gbif_path_tmpl.format(month=month)

    # Resume previous download
    offset = 0 
    if os.path.exists(gbif_path):
        offset = len(pd.read_csv(gbif_path))
    
    end_of_record = False
    limit = 300
    pbar = False
    while not end_of_record:
        # Query the GBIF API
        gbif_parameters = dict(
            continent='NORTH_AMERICA',
            speciesKey='9596413',
            hasCoordinate='true',
            year=2023, month=month,
            limit=limit, offset=offset
        )
        gbif_response = requests.get(gbif_api_endpoint, params=gbif_parameters)
        # Make sure the download went through
        gbif_response.raise_for_status()
        gbif_json = gbif_response.json()
        
        if len(gbif_json['results'])>0:
            # Add results to DataFrame
            pd.DataFrame(gbif_json['results'])[columns].to_csv(
                gbif_path, mode='a',
                index=False, header=(offset==0))

        # Prepare for next loop
        end_of_record = gbif_json['endOfRecords']
        offset += limit
        if not pbar:
            pbar = tqdm.tqdm(total=gbif_json['count'])
            pbar.update(offset)
        pbar.update(limit)

### Load the GBIF data into Python

In [10]:
gbif_paths = []
# Load and format the GBIF data
gbif_dfs = []
for month in range(1, 13):
    # Format the monthly path
    gbif_path = gbif_path_tmpl.format(month=month)

    # Load the data
    gbif_df = pd.read_csv(gbif_path, index_col='key')

    # Select the required columns and append
    gbif_dfs.append(gbif_df[['decimalLatitude', 'decimalLongitude', 'month']])

# Glue the monthly DataFrames together
gbif_df = pd.concat(gbif_dfs)

### Convert the GBIF data to a GeoDataFrame

In [11]:
gbif_gdf = (
    gpd.GeoDataFrame(
        gbif_df, 
        geometry=gpd.points_from_xy(
            gbif_df.decimalLongitude, 
            gbif_df.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['month', 'geometry']]
)
gbif_gdf

### Identify the ecoregion for each observation

In [12]:
gbif_ecoregion_gdf = (
    ecoregions_gdf
    # Match the CRS
    .to_crs(gbif_gdf.crs)
    # Name the index to match the ecoregion index
    .reset_index(names='ecoregion')
    # Find ecoregion for each observation
    .sjoin(
        gbif_gdf.reset_index(names='observation'),
        how='inner', 
        predicate='contains')
    # Select the required columns
    [['ecoregion', 'month', 'name']]
)
gbif_ecoregion_gdf

### Count the observations in each ecoregion

In [13]:
occurrence_df = (
    gbif_ecoregion_gdf
    .groupby(['ecoregion', 'month'])
    .agg(occurrences=('name', 'count'))
)

# Get rid of rare observation noise (possible misidentification?)
occurrence_df = occurrence_df[occurrence_df.occurrences>1]

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    occurrence_df
    .reset_index()
    .groupby(['ecoregion'])
    .mean()
    .occurrences
)
# Take the mean by month
mean_occurrences_by_month = (
    occurrence_df
    .reset_index()
    .groupby(['month'])
    .mean()
    .occurrences
)

# Normalize the observations by the monthly mean throughout the year
occurrence_df['norm_occurrences'] = (
    occurrence_df.occurrences 
    / mean_occurrences_by_ecoregion
    / mean_occurrences_by_month
)
occurrence_df

## Plot the Tasiyagnunpa observations by month

In [14]:
# Prepare a GeoDataFrame for plotting
occurrence_gdf = (
    ecoregions_plot_gdf
    .join(occurrence_df)
    .sort_values('month')
)

# Add a month name column
# occurrence_gdf['Month'] = pd.to_datetime(
#     occurrence_gdf.reset_index().month)#, format='%m')#.dt.month_name()

occurrence_gdf

In [15]:
# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = occurrence_gdf.total_bounds

# Plot occurrence by ecoregion and month
migration_plot = occurrence_gdf.hvplot(
    c='norm_occurrences',
    geo=True, crs=ccrs.Mercator(), tiles='CartoDark',
    title="Tasiyagnunpa migration",
    groupby='month',
    xlim=(xmin, xmax), ylim=(ymin, ymax),
    frame_width=600
)

# Put slider on the bottom
hv.output(widget_location='bottom')

hv.save(migration_plot, 'migration.html')
migration_plot